In [1]:
# Install dependencies
!pip install fastapi uvicorn pyngrok torch torchvision Pillow opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.3 MB/s eta 0:00:00


In [2]:
!pip install python-multipart

In [3]:
import io
import base64
from fastapi import FastAPI, File, UploadFile
from PIL import Image
import torch
import numpy as np
from pydantic import BaseModel
from typing import Dict
import uvicorn
from pyngrok import ngrok
import nest_asyncio

In [8]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import torch
import torch.nn as nn
from fastapi import FastAPI

class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel, self).__init__()
        self.encoder = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.decoder = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

app = FastAPI()

model_path = "/content/drive/MyDrive/mentoring/Kelas-CV/model/best_model.pth"
model = SegmentationModel()

try:
    # Load model weights with strict=False to handle mismatches
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    print("Model loaded successfully.")
except RuntimeError as e:
    print(f"Error loading model: {e}")
except FileNotFoundError:
    print(f"Model file not found at {model_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

<ipython-input-12-b25688d73890>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.device('cpu'))


Model loaded successfully.


In [13]:
# Function to convert PIL image to base64
def pil_to_base64(image: Image) -> str:
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()

# Function to preprocess image for the model
def preprocess_image(image: Image) -> torch.Tensor:
    # Convert image to numpy and perform necessary transformations here
    img = np.array(image) / 255.0  # Normalization example, modify according to your model needs
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float()  # Convert to torch.Tensor
    return img

# Function to postprocess model output into mask
def postprocess_mask(mask: torch.Tensor) -> Image:
    mask = mask.squeeze().detach().numpy()
    mask = (mask * 255).astype(np.uint8)  # Rescale the mask to 0-255 range
    return Image.fromarray(mask)

In [15]:
# Define a model input/output schema
class ImageInput(BaseModel):
    image_base64: str

class ImageOutput(BaseModel):
    segmented_image_base64: str

# API endpoint for image segmentation
@app.post("/segment-image/", response_model=ImageOutput)
async def segment_image(file: UploadFile = File(...)):
    # Open the image file
    image = Image.open(io.BytesIO(await file.read()))

    # Preprocess the image for the model
    input_tensor = preprocess_image(image)

    # Perform model inference
    with torch.no_grad():
        output_mask = model(input_tensor)  # Run inference

    # Postprocess the output mask
    mask_image = postprocess_mask(output_mask)

    # Convert mask to base64 format
    mask_base64 = pil_to_base64(mask_image)

    # Return the segmented mask as base64
    return {"segmented_image_base64": mask_base64}

@app.get("/")
def root():
    return {"message": "Semantic Segmentation API is running!"}

In [16]:
# Tambahkan token ngrok Anda
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
from threading import Thread

# Run the server in a separate thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI server
thread = Thread(target=run)
thread.start()

Public URL: NgrokTunnel: "https://6d63-35-192-21-172.ngrok-free.app" -> "http://localhost:8000"
